# Question 1

In [ ]:
#code for question 1
from types import SimpleNamespace

import numpy as np
from scipy import optimize

import pandas as pd 
import matplotlib.pyplot as plt

class HouseholdSpecializationModelClass:

    def __init__(self):
        """ setup model """

        # a. create namespaces
        par = self.par = SimpleNamespace()
        sol = self.sol = SimpleNamespace()

        # b. preferences
        par.rho = 2.0
        par.nu = 0.001
        par.epsilon = 1.0
        par.omega = 0.5 

        # c. household production
        par.alpha = 0.5
        par.sigma = 1.0

        # d. wages
        par.wM = 1.0
        par.wF = 1.0
        par.wF_vec = np.linspace(0.8,1.2,5)

        # e. targets
        par.beta0_target = 0.4
        par.beta1_target = -0.1

        # f. solution
        sol.LM_vec = np.zeros(par.wF_vec.size)
        sol.HM_vec = np.zeros(par.wF_vec.size)
        sol.LF_vec = np.zeros(par.wF_vec.size)
        sol.HF_vec = np.zeros(par.wF_vec.size)

        sol.beta0 = np.nan
        sol.beta1 = np.nan

    def calc_utility(self,LM,HM,LF,HF):
        """ calculate utility """

        par = self.par
        sol = self.sol

        # a. consumption of market goods
        C = par.wM*LM + par.wF*LF

        # b. home production
        #for this it depends on the values of the epsilon
        if par.sigma == 0:
            H = np.fmin(LM,HM)
        elif par.sigma == 1:
            H = HM**(1-par.alpha) * HF**par.alpha   
            Q = C**par.omega * H**(1-par.omega)
        else:
            H = ((1-par.alpha)*HM**((par.sigma-1)/par.sigma) + par.alpha*HF**((par.sigma-1)/par.sigma))**(par.sigma/(par.sigma-1))
            Q = C**par.omega * H**(1-par.omega)


        # c. total consumption utility
        #Q = C**par.omega*H**(1-par.omega)
        utility = np.fmax(Q,1e-8)**(1-par.rho)/(1-par.rho)

        # d. disutlity of work
        epsilon_ = 1+1/par.epsilon
        TM = LM+HM
        TF = LF+HF
        disutility = par.nu*(TM**epsilon_/epsilon_+TF**epsilon_/epsilon_)
        
        return utility - disutility
            
    def solve_discrete(self,do_print=False):
        """ solve model discretely """
        
        par = self.par
        sol = self.sol
        opt = SimpleNamespace()
        
        # a. all possible choices
        x = np.linspace(0,24,49)
        LM,HM,LF,HF = np.meshgrid(x,x,x,x) # all combinations
    
        LM = LM.ravel() # vector
        HM = HM.ravel()
        LF = LF.ravel()
        HF = HF.ravel()

        # b. calculate utility
        u = self.calc_utility(LM,HM,LF,HF)
    
        # c. set to minus infinity if constraint is broken
        I = (LM+HM > 24) | (LF+HF > 24) # | is "or"
        u[I] = -np.inf
    
        # d. find maximizing argument
        j = np.argmax(u)
        
        opt.LM = LM[j]
        opt.HM = HM[j]
        opt.LF = LF[j]
        opt.HF = HF[j]

        # e. print
        if do_print:
            for k,v in opt.__dict__.items():
                print(f'{k} = {v:6.4f}')

        return opt



# Answer

In [ ]:
sol1 = HouseholdSpecializationModelClass()
new_sigma = np.linspace(0.5,1.5,3)
new_alpha = np.linspace(0.25,0.75,3)

for sigma in range(len(new_sigma)): #this is to consider all the cases of sigma
    for alpha in range(len(new_alpha)): #this is for each sigma ther're mutliple alphas
        sol1.par.sigma = new_sigma[sigma]
        sol1.par.alpha = new_alpha[alpha]

        opt = sol1.solve_discrete() #we solve based on the utility func with the previous parameters 

        if opt.HM == 0: #given that the denominator can't be 0
            answer = np.inf
        else:
            answer = opt.HF / opt.HM #we get our final answer

        print(f'for sigma -> {new_sigma[sigma]} and alpha -> {new_alpha[alpha]} the rate is: {answer:2f}')

# Question 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

model = HouseholdSpecializationModelClass()

log_HF_HM = []
rate_w = []

for i, wF in enumerate(model.par.wF_vec): #this loop is to get all the different values for the salaries
    rate = wF / model.par.wM
    model.par.wF = wF
    opt = model.solve_discrete() #we solve based on the utility func with the previous parameters
    HF = opt.HF
    HM = opt.HM
    log_HF_HM.append(np.log10(HF / HM)) #final answer in logs for the graph
    rate_w.append(np.log10(rate))

print(f'our vector of the rate for both salaries is as follows {rate_w}')
print(f'our vector of the rate for both hours worked at home is as follows {log_HF_HM}')
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(1,1,1)

ax.plot(log_HF_HM, rate_w)

ax.set_title('value of choice')
ax.set_xlabel('log(HF/HM)')
ax.set_ylabel('log(wF/wM)');

# Question 3

In [ ]:
from types import SimpleNamespace

import numpy as np
from scipy import optimize

import pandas as pd 
import matplotlib.pyplot as plt

class HouseholdSpecializationModelClass2:

    def __init__(self):
        """ setup model """

        # a. create namespaces
        par = self.par = SimpleNamespace()
        sol = self.sol = SimpleNamespace()

        # b. preferences
        par.rho = 2.0
        par.nu = 0.001
        par.epsilon = 1.0
        par.omega = 0.5 

        # c. household production
        par.alpha = 0.5
        par.sigma = 1

        # d. wages
        par.wM = 1.0
        par.wF = 1.0
        par.wF_vec = np.linspace(0.8,1.2,5)

        # e. targets
        par.beta0_target = 0.4
        par.beta1_target = -0.1

        # f. solution
        sol.LM_vec = np.zeros(par.wF_vec.size)
        sol.HM_vec = np.zeros(par.wF_vec.size)
        sol.LF_vec = np.zeros(par.wF_vec.size)
        sol.HF_vec = np.zeros(par.wF_vec.size)

        sol.beta0 = np.nan
        sol.beta1 = np.nan

    def calc_utility(self,LM,HM,LF,HF):
        """ calculate utility """

        par = self.par
        sol = self.sol

        # a. consumption of market goods
        C = par.wM*LM + par.wF*LF

        # b. home production
        #for this it depends on the values of the epsilon
        if par.sigma == 0:
            H = np.fmin(HF,HM)
        elif par.sigma == 1:
            H = HM**(1-par.alpha) * HF**par.alpha   
            Q = C**par.omega * H**(1-par.omega)
        else:
            H = ((1-par.alpha)*HM**((par.sigma-1)/par.sigma) + par.alpha*HF**((par.sigma-1)/par.sigma))**(par.sigma/(par.sigma-1))
            Q = C**par.omega * H**(1-par.omega)


        # c. total consumption utility
        #Q = C**par.omega*H**(1-par.omega)
        utility = np.fmax(Q,1e-8)**(1-par.rho)/(1-par.rho)

        # d. disutlity of work
        epsilon_ = 1+1/par.epsilon
        TM = LM+HM
        TF = LF+HF
        disutility = par.nu*(TM**epsilon_/epsilon_+TF**epsilon_/epsilon_)
        
        return utility - disutility

    def solve_discrete(self,do_print=False):
        """ solve model discretely """
        
        par = self.par
        sol = self.sol
        opt = SimpleNamespace()
        
        # a. all possible choices
        x = np.linspace(0,24,49)
        LM,HM,LF,HF = np.meshgrid(x,x,x,x) # all combinations
    
        LM = LM.ravel() # vector
        HM = HM.ravel()
        LF = LF.ravel()
        HF = HF.ravel()

        # b. calculate utility
        u = self.calc_utility(LM,HM,LF,HF)
    
        # c. set to minus infinity if constraint is broken
        I = (LM+HM > 24) | (LF+HF > 24) # | is "or"
        u[I] = -np.inf
    
        # d. find maximizing argument
        j = np.argmax(u)
        
        opt.LM = np.log(LM[j])
        opt.HM = np.log(HM[j])
        opt.LF = np.log(LF[j])
        opt.HF = np.log(HF[j])

        # e. print
        if do_print:
            for k,v in opt.__dict__.items():
                print(f'{k} = {v:6.4f}')

        return opt

    def solve_continous(self,do_print=False):
        """ solve model continously """
        par = self.par
        sol = self.sol
        opt = SimpleNamespace()

        #we need to present our objective function that we want to maximize
        def objective_f(x):
            return -self.calc_utility(x[0], x[1], x[2], x[3])
        
        #now we need to presnet our constraints
        constraint1 = lambda x: 24 - x[0] - x[1]
        constraint2 = lambda x: 24 - x[2] - x[3]
        constraints = ({'type': 'ineq', 'fun': constraint1},{'type': 'ineq', 'fun': constraint2})

        #now make the guess of the values
        guess = [12,12,12,12]

        #now create the boundaries for the values 
        bounds = ((0,24),(0,24),(0,24),(0,24))

        #now create the minimize function with the previous parameters
        solution = optimize.minimize(objective_f,guess,method='SLSQP',bounds=bounds,constraints=constraints)
        
        #give the final values to the variables
        opt.LM = solution.x[0]
        opt.HM = solution.x[1]
        opt.LF = solution.x[2]
        opt.HF = solution.x[3]
        
        return opt

    def solve_wF_vec(self,discrete=False):
        """ solve model for vector of female wages """


        pass

    def run_regression(self):
        """ run regression """

        par = self.par
        sol = self.sol

        x = np.log(par.wF_vec)
        y = np.log(sol.HF_vec/sol.HM_vec)
        A = np.vstack([np.ones(x.size),x]).T
        sol.beta0,sol.beta1 = np.linalg.lstsq(A,y,rcond=None)[0]
    
    def estimate(self,alpha=None,sigma=None):
        """ estimate alpha and sigma """

        pass

In [ ]:
sol1 = HouseholdSpecializationModelClass2()
new_sigma = np.linspace(0.5,1.5,3)
new_alpha = np.linspace(0.25,0.75,3)

for sigma in range(len(new_sigma)):
    for alpha in range(len(new_alpha)):
        sol1.par.sigma = new_sigma[sigma]
        sol1.par.alpha = new_alpha[alpha]

        opt = sol1.solve_continous()  

        if opt.HM == 0:
            answer = np.inf
        else:
            answer = (opt.HF) / (opt.HM)

        print(f'for sigma -> {new_sigma[sigma]} and alpha -> {new_alpha[alpha]} the rate is: {answer:2f}')

# Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

model = HouseholdSpecializationModelClass2()

log_HF_HM = []
rate_w = []

for i, wF in enumerate(model.par.wF_vec):
    rate = wF / model.par.wM
    model.par.wF = wF
    opt = model.solve_continous() 
    HF = opt.HF
    HM = opt.HM
    log_HF_HM.append(np.log10(HF / HM))
    rate_w.append(np.log10(rate))

print(f'our vector of the rate for both salaries is as follows {rate_w}')
print(f'our vector of the rate for both hours worked at home is as follows {log_HF_HM}')
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(1,1,1)

ax.plot(log_HF_HM, rate_w)

ax.set_title('value of choice')
ax.set_xlabel('log(HF/HM)')
ax.set_ylabel('log(wF/wM)');